### Custom Thermal Profile Creation

While Rayleigh can define an initial, conductive thermal profile based on the details of internal heating and boundary conditions, alternative ell=0 initial profiles may be desired.  This notebook illustrates how to do this using the rayleigh_spectral_input module.

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from rayleigh_spectral_input import *
from rayleigh_diagnostics import Shell_Avgs

We first define a function that accepts a parameter named radius, assumed to be a numpy array of points at which you wish to compute your custom profile.  The name of this non-optional parameter must be 'radius.'  The function should return f(radius), where f is the custom thermal profile. 

Additional options should be specified as optional keyword parameters.

In [ ]:
def rpower(radius,amp=1,power=1):
    f = amp*radius**power
    return f



Next, we initialize the grid on which we wish to compute this function.  We use the rayleigh_spectral_input module for this.  Since we are generating a spherically symmetric function, ntheta is set to 1.   Nr can be larger or smaller than the grid you plan to use in Rayleigh.   It only needs to be large enough to fully resolve your function f(radius).  The spectral input object will contain the Chebyshev coefficients needed to represent this profile on a Rayleigh Chebyshev grid.

In [ ]:
ntheta = 1
nr = 128
rmin = 0.5
rmax = 1
si = SpectralInput(n_theta=ntheta, n_r=nr)


Next, we define our function parameters via a dictionary whose keys match the names of those optional parameters needed to fully specify our function.

In [ ]:
func_pars = {}
func_pars['amp']=1.0
func_pars['power']=0.5

si.transform_from_rtp_function(rpower, func_kwargs=func_pars, rmin=rmin, rmax=rmax)


print('Coefficients array shape: ', si.coeffs.shape)
fig,ax = plt.subplots()
ax.plot(np.abs(si.coeffs[:,0,0]))
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('Chebyshev Degree n')
ax.set_ylabel('Amplitude')
plt.show()

Finally, we save the profile to a file that can be read in when Rayleigh is initialized.

In [ ]:
si.write('my_custom_profile.dat')

To use this profile, with random thermal perturbations for the non-spherically symmetric components of temperature/entropy, we include the following in our main_input:  

&Initial_Conditions_Namelist  
init_type = 7  
custom_thermal_file = 'my_custom_profile.dat'  
temp_amp = 0.1  
/

We can then check the results by outputting a Shell_Avgs file on timestep 1.  Note that we use a pretty large temp_amp here for illustration purposes.  This let's us easily view the standard deviation about the spherically symmetric mean.  This let's us verify that non-spherically symmetric modes were not set to zero.


In [ ]:
sa = Shell_Avgs('00000001')

In [ ]:
fig, ax = plt.subplots()
thermal = sa.vals[:,0,sa.lut[501],0]
thermal_stdev = np.sqrt(sa.vals[:,1,sa.lut[501],0])
ax.plot(sa.radius,thermal, 'ro',label='T (from Rayleigh)')
ax.plot(sa.radius,func_pars['amp']*sa.radius**func_pars['power'], label='T (supplied)')
ax.plot(sa.radius,thermal+thermal_stdev, color = 'gray')
ax.plot(sa.radius,thermal-thermal_stdev, color = 'gray')
ax.set_xlabel('Radius')
ax.set_ylabel('Temperature')
ax.legend()
plt.show()